In [2]:
import pandas as pd
import json

In [3]:
file1_df = pd.read_json('raw_data/purchase_data.json')
file2_df = pd.read_json('raw_data/purchase_data2.json')
print (len(file1_df))
print (len(file2_df))

780
78


In [4]:
#Check to make sure columns are the same between both files
file1_df.columns,file2_df.columns

(Index(['Age', 'Gender', 'Item ID', 'Item Name', 'Price', 'SN'], dtype='object'),
 Index(['Age', 'Gender', 'Item ID', 'Item Name', 'Price', 'SN'], dtype='object'))

In [5]:
#append contents of file2 to file 1
pymoli_df = file1_df.append(file2_df)
#pymoli_df = pd.merge(file1_df,file2_df, on="", how="inner")

In [6]:
#player count
playerCount = len(pymoli_df['SN'])     #alternate way: playerCount = pymoli_df['SN'].count().mean()
print(playerCount)

858


In [7]:
#purchase analysis

#number of unique items
uniqueItemCount = len(pymoli_df['Item ID'].unique())

#average purchase price
averagePurchasePrice = pymoli_df["Price"].mean()

#total number of purchases
purchaseCount = pymoli_df['Price'].count() #pymoli_df.loc[pymoli_df['Price'] == 0.00] #search through purchases to make sure non are free/0.00

#total revenue
totalRevenue = pymoli_df['Price'].sum()
#pymoli_df['Price'].value_counts()
#summary_df = pd.DataFrame({"UniqueItems":uniqueItemCount})

#summary_df["test"] = averagePurchasePrice
price_df_summary = pd.DataFrame({'Unique Item Count':uniqueItemCount,'Average Purchase Price':averagePurchasePrice,'Purchase Count':purchaseCount,'Total Revenue':totalRevenue},index=[0])
price_df_summary.head()

,Average Purchase Price,Purchase Count,Total Revenue,Unique Item Count
0,2.930571,858,2514.43,184


In [8]:
#gender demographics
genderCount = pymoli_df['Gender'].value_counts()
percentageGender = pymoli_df['Gender'].value_counts()/len(pymoli_df)*100
genderDemo_summary_df = pd.DataFrame()
genderDemo_summary_df["Gender Count"] = genderCount
genderDemo_summary_df["Percentage Gender"] = percentageGender

In [9]:
genderDemo_summary_df.head()

,Gender Count,Percentage Gender
Male,697,81.235431
Female,149,17.365967
Other / Non-Disclosed,12,1.398601


In [10]:
#purchase analysis (gender-specific)
gender_df_price =  pymoli_df.groupby(['Gender']).mean()["Price"]
gender_df_count =  pymoli_df.groupby(['Gender']).count()["Price"]

#normalized totals = divide total purchase value by count of each gender in dataset
gender_df_normalized_total=gender_df_price/gender_df_count
gender_df_normalized_total_percentage = gender_df_normalized_total * 100

genderPurchase_summary_df = pd.DataFrame({'Average Purchase Price':gender_df_price,'Purchase Count':gender_df_count,'Normalized Totals':gender_df_normalized_total,'Normalized Totals %':gender_df_normalized_total_percentage})

In [11]:
genderPurchase_summary_df.head()

,Average Purchase Price,Normalized Totals,Normalized Totals %,Purchase Count
Gender,,,,
Female,2.847584,0.019111,1.911130,149
Male,2.944448,0.004224,0.422446,697
Other / Non-Disclosed,3.155000,0.262917,26.291667,12


In [12]:
#age demographics

bins = [0,9,14,19,24,29,34,39,100] 
bin_names =["<10", "10-14", "15-19","20-24","25-29", "30-34","35-39", "40+"]
pymoli_df["Age Demographics"]= pd.cut(pymoli_df["Age"], bins, labels=bin_names)
pymoli_df.head()

,Age,Gender,Item ID,Item Name,Price,SN,Age Demographics
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34,35-39
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46,20-24
2,34,Male,174,Primitive Blade,2.46,Assastnya25,30-34
3,21,Male,92,Final Critic,1.36,Pheusrical25,20-24
4,23,Male,63,Stormfury Mace,1.27,Aela59,20-24


In [13]:
ageAveragePurchasePrice = pymoli_df.groupby(["Age Demographics"]).mean()['Price']
agePurchaseCount = pymoli_df.groupby(["Age Demographics"]).count()['Age']
agePurchaseValueTotal= pymoli_df.groupby(["Age Demographics"]).sum()['Price']

#normalized totals = divide total purchase value by count of each gender in dataset
age_df_normalized_total=ageAveragePurchasePrice/agePurchaseCount
age_df_normalized_total_percentage = age_df_normalized_total * 100
age_summary_df = pd.DataFrame({'Average Purchase Price':ageAveragePurchasePrice,'Purchase Count':agePurchaseCount,'Normalized Totals':age_df_normalized_total,'Normalized Totals %':age_df_normalized_total_percentage,"Total Purchase Value": agePurchaseValueTotal})

In [14]:
#Reorder columns for age_summary (demographics) output:
age_summary_df = age_summary_df[['Purchase Count','Average Purchase Price','Normalized Totals','Normalized Totals %', 'Total Purchase Value']]
age_summary_df.head(10)

,Purchase Count,Average Purchase Price,Normalized Totals,Normalized Totals %,Total Purchase Value
Age Demographics,,,,,
10-14,38,2.787105,0.073345,7.334488,105.91
15-19,144,2.894653,0.020102,2.010176,416.83
20-24,372,2.923817,0.007860,0.785972,1087.66
25-29,134,2.958507,0.022078,2.207841,396.44
30-34,71,2.973803,0.041885,4.188455,211.14
35-39,48,2.932708,0.061098,6.109809,140.77
40+,18,3.244444,0.180247,18.024691,58.40
<10,33,2.947879,0.089330,8.932966,97.28


In [15]:
#Top spenders
TopSpendersAveragePurchasePrice = pymoli_df.groupby(["SN"]).mean()['Price']
TopSpendersPurchaseCount = pymoli_df.groupby(["SN"]).count()['Age']
TopSpendersPurchaseValueTotal= pymoli_df.groupby(["SN"]).sum()['Price']
top_spend_summary_df = pd.DataFrame({'Average Purchase Price':TopSpendersAveragePurchasePrice,'Purchase Count':TopSpendersPurchaseCount,"Total Purchase Value": TopSpendersPurchaseValueTotal})
top_spend_summary_df = top_spend_summary_df.sort_values(['Total Purchase Value'], axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')
#top_spend_summary_df=top_spend_summary_df.head(5)
top_spend_summary_df = top_spend_summary_df[0:5]  
top_spend_summary_df.head()

,Average Purchase Price,Purchase Count,Total Purchase Value
SN,,,
Undirrala66,3.412,5,17.06
Aerithllora36,3.775,4,15.10
Saedue76,3.390,4,13.56
Sondim43,3.255,4,13.02
Mindimnya67,3.185,4,12.74


In [66]:
#Most popular items
mpiPurchasePrice = pymoli_df.groupby(["Item Name","Item ID"]).mean()['Price']
mpiPurchaseCount = pymoli_df.groupby(["Item Name","Item ID"]).count()['Price']
mpiPurchaseValueTotal= pymoli_df.groupby(["Item Name","Item ID"]).sum()['Price']
mpi_df = pd.DataFrame({'Purchase Price':mpiPurchasePrice,'Purchase Count':mpiPurchaseCount,"Total Purchase Value": mpiPurchaseValueTotal})
mpi_summary_df = mpi_df.sort_values(['Purchase Count'], axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')
mpi_df = mpi_df[0:5]  


In [67]:
mpi_summary_df.head()

,,Purchase Count,Purchase Price,Total Purchase Value
Item Name,Item ID,,,
Arcane Gem,84,12,2.445000,29.34
"Betrayal, Whisper of Grieving Widows",39,11,2.350000,25.85
Trickster,31,10,2.322000,23.22
Feral Katana,154,9,2.616667,23.55
Serenity,13,9,1.490000,13.41


In [68]:
#Most profitable items
mpi2PurchasePrice = pymoli_df.groupby(["Item Name","Item ID"]).mean()['Price']
mpi2PurchaseCount = pymoli_df.groupby(["Item Name","Item ID"]).count()['Price']
mpi2PurchaseValueTotal= pymoli_df.groupby(["Item Name","Item ID"]).sum()['Price']
mpi2_df = pd.DataFrame({'Purchase Price':mpi2PurchasePrice,'Item Price':itemPrice,'Purchase Count':mpi2PurchaseCount,"Total Purchase Value": mpi2PurchaseValueTotal})
mpi2_summary_df = mpi_df.sort_values(['Total Purchase Value'], axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')
mpi2_df = mpi_df[0:5]  

In [69]:
mpi2_summary_df.head()

,,Purchase Count,Purchase Price,Total Purchase Value
Item Name,Item ID,,,
"Alpha, Oath of Zeal",79,8,2.68375,21.47
"Aetherius, Boon of the Blessed",137,4,4.75000,19.00
Alpha,130,7,1.56000,10.92
Agatha,120,5,1.91000,9.55
Abyssal Shard,162,3,2.04000,6.12
